In [1]:
def Prepro(df_train):
    return df_train
def encoding(df_train):
    X = []
    if 'target_1' in df_train.columns:
        Y1 = df_train['target_1'].values
        Y2 = df_train['target_2'].values
        del df_train['target_2']
        del df_train['target_1']
        del df_train['cm_key']
        df_tr = df_train[df_train.columns[:43]]
        X = df_tr.to_dict('records')
        from sklearn.feature_extraction import DictVectorizer
        enc = DictVectorizer()
        Xenc =enc.fit_transform(X)
        return [Xenc,Y1,Y2]
    else:
        del df_train['cm_key']
        df_tr = df_train[df_train.columns[:43]]
        X = df_tr.to_dict('records')
        from sklearn.feature_extraction import DictVectorizer
        enc = DictVectorizer()
        Xenc =enc.fit_transform(X)
        return [Xenc]

def encoding2(df_train):
    X = []
    if 'target_1' in df_train.columns:
        Y1 = df_train['target_1'].values
        Y2 = df_train['target_2'].values
        del df_train['target_2']
        del df_train['target_1']
        del df_train['cm_key']
        df_tr = df_train[df_train.columns[:43]]
        X = df_tr.to_dict('records')
        from sklearn.feature_extraction import DictVectorizer
        enc = DictVectorizer()
        Xenc =enc.fit_transform(X)
        return [Xenc,Y1,Y2]
    else:
        df_tr = df_train[df_train.columns[:43]]
        X = df_tr.to_dict('records')
        from sklearn.feature_extraction import DictVectorizer
        enc = DictVectorizer()
        Xenc =enc.fit_transform(X)
        return [Xenc]    
    
    
import pandas
df_train = pandas.read_csv('AMTRAIN.csv')#pandas.read_csv('AMTRAIN.csv')
df_valid = pandas.read_csv('AMVALID.csv')
Y_TEST_1 = df_valid['target_1']
Y_TEST_2 = df_valid['target_2']
del df_valid['target_1']
del df_valid['target_2']
Key1 = df_valid['cm_key'].values

cols = [
 'cm_key',
 'mvar1',
 'mvar2',
 'mvar4',
 'mvar5',
 'mvar6',
 'mvar7',
 'mvar8',
 'mvar10',
 'mvar11',
 'mvar12',
 'mvar13',
 'mvar14',
 'mvar15',
 'mvar16',
 'mvar17',
 'mvar18',
 'mvar19',
 'mvar20',
 'mvar21',
 'mvar22',
 'mvar23',
 'mvar24',
 'mvar25',
 'mvar26',
 'mvar27',
  'mvar28',
 'mvar29',
 'mvar30',
 'mvar31',
 'mvar32',
 'mvar33',
 'mvar34',
 'mvar35',
 'mvar36',
 'mvar37',
 'mvar38',
 'mvar39',
#  'mvar40',
#  'mvar41',
#  'mvar42',
#  'mvar43',
#  'mvar44',
#  'mvar45',
       ]

df_train = df_train[cols+[ 'target_1', 'target_2']]
df_valid = df_valid[cols]
#df_test = df_test[cols]

df_train = Prepro(df_train)
df_valid = Prepro(df_valid)
train = encoding(df_train)
valid = encoding(df_valid)
X_tr = train[0]
Y_tr_1  = train[1]
Y_tr_2  = train[2]
X_va = valid[0]


# #Feature Union
Y = Y_tr_2
# pca = PCA(n_components=3)
# selection = SelectKBest(k=25)
# modelused = FeatureUnion([("pca", pca), ("univ_select", selection)])
# X_TR = modelused.fit(X_tr.toarray(), Y).transform(X_tr.toarray())
# X_VA = modelused.fit(X_tr.toarray(), Y).transform(X_va.toarray())

X_TR = X_tr
X_VA = X_va

In [123]:
clf = AdaBoostClassifier()
clf.fit(X_tr,Y_tr_1)
from sklearn.calibration import CalibratedClassifierCV

sig_clf = CalibratedClassifierCV(clf, method="sigmoid", cv="prefit")
sig_clf.fit(X_tr.toarray(), Y_tr_1)

print confusion_matrix(Y_tr_1, sig_clf.predict(X_tr.toarray()))

[[30776   346]
 [ 8495   383]]


In [124]:
df_valid['proba'] = sig_clf.predict_proba(X_VA.toarray()).T[1]
df_valid['KEY'] = Key1
df_valid = df_valid.sort_values(['proba'], ascending=False)
df_valid2 = df_valid[:1000]
Key2 = df_valid2['KEY'].values
del df_valid2['KEY']
del df_valid2['proba']

df_train['Tar'] = Y_tr_2
df_train['T'] = Y_tr_1
df_train2 = df_train[df_train['T'] ==1]
Y_2  = df_train2['Tar'].values

del df_train2['T']
del df_train2['Tar']

del df_train2['mvar12']
del df_valid2['mvar12']

train = encoding2(df_train2)
valid = encoding2(df_valid2)
X_tr2 = train[0]
X_va2 = valid[0]

ValueError: Length of values does not match length of index

In [125]:
clf = GradientBoostingClassifier(min_samples_leaf=100)
clf.fit(X_tr2,Y_2)


GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=100,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=100, presort='auto', random_state=None,
              subsample=1.0, verbose=0, warm_start=False)

In [126]:
df_valid2['KEY'] = Key2
df_valid2['pred'] = clf.predict(X_va2.toarray())

# maxproba = []
# for i in clf.predict_proba(X_va2.toarray()):
#     maxproba.append(max(i))
# df_valid2['M']  = maxproba
# df_valid2 = df_valid2.sort_values(['M'], ascending=False)
df_valid = df_valid2[['KEY','pred']]
df_valid.to_csv('kowshik_pred16.csv',index=False,index_label=False,header=False)

<1000x72 sparse matrix of type '<type 'numpy.float64'>'
	with 24576 stored elements in Compressed Sparse Row format>

In [122]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier,OneVsOneClassifier,OutputCodeClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn import svm
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest,chi2,SelectKBest,SelectFromModel
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.multiclass import OneVsRestClassifier,OutputCodeClassifier
from sklearn.metrics import confusion_matrix,accuracy_score
from collections import Counter
from sklearn.feature_extraction import DictVectorizer